In [1]:
import glob
import dask
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
from dask.distributed import Client
from dask_jobqueue import PBSCluster

walltime = '02:00:00'
cores = 48
memory = '192GB'
cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory),
                     job_extra=['-l ncpus='+str(cores),
                                '-l mem='+str(memory),
                                '-P xv83',
                                '-l jobfs=100GB',
                                '-l storage=gdata/xv83'],
                     local_directory='$PBS_JOBFS',
                     # env_extra=['export MALLOC_TRIM_THRESHOLD_="0"'],
                     header_skip=["select"])

/g/data/xv83/ds0092/software/miniconda3/envs/pangeo/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
cluster.scale(jobs=1)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.6.74.65:8787/status,
Dashboard: http://10.6.74.65:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.74.65:38211,Workers: 0
Dashboard: http://10.6.74.65:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Load some CAFE forecasts

In [4]:
TIME_NAME = 'time'
INIT_DATE_NAME = 'init_date'
LEAD_TIME_NAME = 'lead_time'

def open_zarr_forecasts(files, convert_time_to_lead=True, preprocess=None, **open_kwargs):
    def convert_time_to_lead(ds):
        init_date = ds[TIME_NAME].values[0]
        lead_time = range(len(ds[TIME_NAME]))
        aux_coord = ds[TIME_NAME].rename({TIME_NAME: LEAD_TIME_NAME}
            ).assign_coords({LEAD_TIME_NAME: lead_time})
        dataset = ds.rename({TIME_NAME: LEAD_TIME_NAME}
            ).assign_coords({LEAD_TIME_NAME: lead_time,
                             INIT_DATE_NAME: init_date})
        return aux_coord, dataset
        
    open_tasks = [dask.delayed(xr.open_zarr)(f, **open_kwargs) for f in files]
    preprocess_tasks = [
        dask.delayed(preprocess)(task) for task in open_tasks] if preprocess is not None else open_tasks
    convert_tasks = [dask.delayed(convert_time_to_lead)(task) for task in preprocess_tasks]
    
    aux_coords, datasets = zip(*dask.compute(convert_tasks)[0])
    aux_coord = xr.concat(aux_coords, dim=INIT_DATE_NAME, 
                          coords='minimal', compat='override')  
    dataset = xr.concat(datasets, dim=INIT_DATE_NAME, 
                        coords='minimal',compat='override')
    return dataset.assign_coords({TIME_NAME: aux_coord})

In [5]:
data_dir = '/g/data/xv83/dcfp/CAFE-f6'
realm = 'atmos_isobaric_month.zarr.zip'

In [6]:
files = sorted(glob.glob(f'{data_dir}/c5-d60-pX-f6-*/{realm}'))

In [7]:
def truncate_latitudes(ds):
    for dim in ds.dims:
        if 'lat' in dim:
            ds = ds.assign_coords({dim: ds[dim].round(decimals=10)})
    return ds

atmos_month = open_zarr_forecasts(files, preprocess=truncate_latitudes)

In [8]:
atmos_month

<xarray.Dataset>
Dimensions:         (init_date: 80, lead_time: 120, ensemble: 96, level: 21, lat: 90, lon: 144)
Coordinates:
    area            (lat, lon) float32 dask.array<chunksize=(90, 144), meta=np.ndarray>
    average_DT      (lead_time) timedelta64[ns] dask.array<chunksize=(120,), meta=np.ndarray>
    average_T1      (lead_time) datetime64[ns] dask.array<chunksize=(120,), meta=np.ndarray>
    average_T2      (lead_time) datetime64[ns] dask.array<chunksize=(120,), meta=np.ndarray>
  * ensemble        (ensemble) int64 1 2 3 4 5 6 7 8 ... 89 90 91 92 93 94 95 96
  * lat             (lat) float64 -89.49 -87.98 -85.96 ... 85.96 87.98 89.49
  * level           (level) float32 1.0 5.0 10.0 20.0 ... 850.0 925.0 1e+03
  * lon             (lon) float64 1.25 3.75 6.25 8.75 ... 353.8 356.2 358.8
  * lead_time       (lead_time) int64 0 1 2 3 4 5 6 ... 114 115 116 117 118 119
    zsurf           (lat, lon) float32 dask.array<chunksize=(90, 144), meta=np.ndarray>
  * init_date       (init_date) object 1981-05-16 12:00:00 ... 2020-11-16 00:...
    time            (init_date, lead_time) object 1981-05-16 12:00:00 ... 203...
Data variables: (12/38)
    DELP            (init_date, lead_time, ensemble, level, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 1, 90, 144), meta=np.ndarray>
    awp             (init_date, lead_time, ensemble, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 90, 144), meta=np.ndarray>
    evap            (init_date, lead_time, ensemble, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 90, 144), meta=np.ndarray>
    h500            (init_date, lead_time, ensemble, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 90, 144), meta=np.ndarray>
    hght            (init_date, lead_time, ensemble, level, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 1, 90, 144), meta=np.ndarray>
    lwdn_sfc        (init_date, lead_time, ensemble, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 90, 144), meta=np.ndarray>
    ...              ...
    u_ref           (init_date, lead_time, ensemble, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 90, 144), meta=np.ndarray>
    ucomp           (init_date, lead_time, ensemble, level, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 1, 90, 144), meta=np.ndarray>
    v_ref           (init_date, lead_time, ensemble, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 90, 144), meta=np.ndarray>
    vcomp           (init_date, lead_time, ensemble, level, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 1, 90, 144), meta=np.ndarray>
    vis_exopd_vl_c  (init_date, lead_time, ensemble, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 90, 144), meta=np.ndarray>
    wvp             (init_date, lead_time, ensemble, lat, lon) float32 dask.array<chunksize=(1, 24, 96, 90, 144), meta=np.ndarray>
Attributes:
    comment:    pressure level interpolator, version 3.0, precision=double
    filename:   atmos_isobaric_month.zarr
    grid_tile:  N/A
    grid_type:  regular
    title:      AccessOcean-AM2

In [9]:
GMST = atmos_month['t_surf'].weighted(atmos_month['area']).mean(['lat','lon']).compute()

In [10]:
GMST_anom = GMST.groupby('init_date.month').map(lambda x: x - x.mean(['init_date', 'ensemble']))